## ADES demonstration - OGC API Processes 

Deploy a CWL application package as a WPS processing service and submit an execution request

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import requests
import os
import getpass
import json
import yaml
from time import sleep
from helpers2 import *

## Deploy the process

In [14]:
token = getpass.getpass('Token:')

Token: ·


In [15]:
cwl_url = 'https://raw.githubusercontent.com/terradue-ogc-tb16/application-chaining/master/processes/vegetation-index.cwl'

In [16]:
r = deploy_process(cwl_url, token)

print(r.status_code, r.reason, r.url)

201 Created http://ades-dev.eoepca.terradue.com/terradue/wps3/processes


## /processes

In [19]:
r = get_process(token)

print(r.status_code, r.reason, r.url)

200 OK http://ades-dev.eoepca.terradue.com/terradue/wps3/processes


In [21]:
r.json()

{'processes': [{'id': 'eoepcaadesundeployprocess',
   'title': 'Eoepca Deploy Process',
   'abstract': 'This method will undeploy an application encapsulated within a Docker container as a process accessible through the WPS interface.',
   'version': '1.0.0',
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'canonical',
     'type': 'application/json',
     'title': 'Process Description',
     'href': '/watchjob/processes/eoepcaadesundeployprocess/'}]},
  {'id': 'nb_vegetation_index_',
   'title': 'Vegetation index',
   'abstract': 'Vegetation index',
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'canonical',
     'type': 'application/json',
     'title': 'Process Description',
     'href': '/watchjob/processes/nb_vegetation_index_/'}]},
  {'id': 'GetStatus',
   'title': 'Produce an updated ExecuteRespon

## /processes/{id}

In [23]:
process_id = 'vegetation_index_'

r = get_process(token, process_id)

print(r.status_code, r.reason, r.url)

200 OK http://ades-dev.eoepca.terradue.com/terradue/wps3/processes/vegetation_index_


In [24]:
r.json()

{'process': {'id': 'vegetation_index_',
  'title': 'Vegetation index',
  'abstract': 'Vegetation index',
  'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
  'outputTransmission': ['value', 'reference'],
  'links': [{'rel': 'canonical',
    'type': 'application/json',
    'title': 'Execute End Point',
    'href': '/watchjob/processes/vegetation_index_/jobs/'}],
  'inputs': [{'id': 'input_reference',
    'title': 'Sentinel-2 Level-2 catalog reference',
    'abstract': 'Sentinel-2 Level-2 acquisition catalog reference',
    'minOccurs': '1',
    'maxOccurs': '999',
    'input': {'literalDataDomains': [{'dataType': {'name': 'string'},
       'valueDefinition': {'anyValue': True}}]}},
   {'id': 'aoi',
    'title': 'Area of interest',
    'abstract': 'Area of interest in WKT',
    'minOccurs': '1',
    'maxOccurs': '1',
    'input': {'literalDataDomains': [{'dataType': {'name': 'string'},
       'valueDefinition': {'anyValue': True}}]}}],
  'outputs': [{'id': 'wf_outputs',

List the inputs

In [26]:
inputs = r.json()['process']['inputs']

inputs

[{'id': 'input_reference',
  'title': 'Sentinel-2 Level-2 catalog reference',
  'abstract': 'Sentinel-2 Level-2 acquisition catalog reference',
  'minOccurs': '1',
  'maxOccurs': '999',
  'input': {'literalDataDomains': [{'dataType': {'name': 'string'},
     'valueDefinition': {'anyValue': True}}]}},
 {'id': 'aoi',
  'title': 'Area of interest',
  'abstract': 'Area of interest in WKT',
  'minOccurs': '1',
  'maxOccurs': '1',
  'input': {'literalDataDomains': [{'dataType': {'name': 'string'},
     'valueDefinition': {'anyValue': True}}]}}]

## Execute - /processes/{id}/jobs

In [27]:
execute_inputs = [{'id': 'input_reference',
                               'input': {'dataType': {'name': 'string'},
                                         'value': 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808'}},
                              {'id': 'aoi',
                               'input': {'dataType': {'name': 'string'},
                                         'value': 'POLYGON((136.112726861895 -36.227897298303,137.333826362695 -36.2103069464338,137.305145407058 -35.2211228310596,136.099040812374 -35.2380875358202,136.112726861895 -36.227897298303))'}}]

In [30]:
r = execute_process(token, process_id, execute_inputs)

print(r.status_code, r.reason, r.url)

201 Created http://ades-dev.eoepca.terradue.com/terradue/wps3/processes/vegetation_index_/jobs


In [33]:
job_location = r.headers['Location']

job_location

'/terradue/watchjob/processes/vegetation_index_/jobs/873c698a-1f7d-11eb-ac9a-0a580a8105ca'

## Poll /processes/{id}/jobs/{job_id}

In [39]:
r = get_status(token, job_location)

print(r.status_code, r.reason, r.url)

200 OK http://ades-dev.eoepca.terradue.com/terradue/watchjob/processes/vegetation_index_/jobs/873c698a-1f7d-11eb-ac9a-0a580a8105ca


In [41]:
r.json()

{'jobID': '873c698a-1f7d-11eb-ac9a-0a580a8105ca',
 'status': 'running',
 'message': 'running',
 'progress': '0',
 'links': [{'href': 'http://ades-dev.eoepca.terradue.com/terradue/watchjob/processes/vegetation_index_/jobs/873c698a-1f7d-11eb-ac9a-0a580a8105ca',
   'rel': 'self',
   'type': 'application/json',
   'title': 'get Status'}]}

In [44]:

while r.json()['status'] == 'running':

    r = get_status(token, job_location)

    if r.json()['status'] == 'failed': 
    
        print(r.json())
        
        break
        
    if r.json()['status'] == 'successful':  

        print( r.json()['links'][0]['href'])
        
        break
    
    else:
        
        print('Polling - {}'.format(r.json()['status']))
        
        sleep(30)
        

Polling - running


KeyboardInterrupt: 

## Results - /processes/{id}/jobs/{job_id}/result

In [ ]:
response

In [48]:
r = get_result(token, job_location)

print(r.status_code, r.reason, r.url)

500 The result file is not ready http://ades-dev.eoepca.terradue.com//terradue/watchjob/processes/vegetation_index_/jobs/873c698a-1f7d-11eb-ac9a-0a580a8105ca/result


In [ ]:
response = r.json()

response

In [ ]:
results = json.loads(response['outputs'][0]['value']['inlineValue'])

results

## Inspect the results

Use the STAC catalog endpoint in the "Exploiting the result.ipynb" Notebook 

In [ ]:
stac_catalog_endpoint = results['stac:catalog']['href']

stac_catalog_endpoint